<span style="color:red; font-size:3em">Сначала посмотри в файл **idea.ipynb**</span>

In [1]:
%load_ext autoreload
%autoreload 2

# Визуализация задачи

In [2]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash_core_components as dcc
import dash_html_components as html
from dash import no_update
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash

from is_jupyterlab_running import is_jupyterlab_running
from problem import Problem, Vector, VectorRegressor
from problem.figures import Cylinder

import traceback

In [3]:
r = 100
h = 200
d = 100
problem = Problem(r=r, d=d, h=h)

In [40]:
cylinder = Cylinder(r=r, h=h, d=d)
start_c_vector = problem.get_c(x=0, z=h / 2)
start_ray = problem.get_ray(c=start_c_vector)


xOy = go.Surface(
    x=np.outer(np.ones(100), np.linspace(-4 * r, 4 * r, 100)),
    y=np.outer(np.linspace(-4 * r, 2 * r + 2 * d, 100), np.ones(100)),
    z=np.outer(np.zeros(100), np.zeros(100)),
)

y_eq_r = go.Surface(
    x=np.outer(np.ones(100), np.linspace(-4 * r, 4 * r, 100)),
    y=r * np.outer(np.ones(100), np.ones(100)),
    z=np.outer(np.linspace(0, h, 100), np.ones(100)),
    opacity=0.7,
    colorscale=[[0, "orange"], [1, "orange"]],
)

fig_layout = dict(
    scene=dict(
        xaxis=dict(nticks=4, range=[-4 * r, 4 * r],),
        yaxis=dict(nticks=4, range=[-4 * r, 2 * r + 2 * d],),
        zaxis=dict(nticks=4, range=[-h, h],),
        aspectratio=dict(x=1, y=1, z=1),
    ),
    height=1000,
)

fig = go.Figure(
    data=[cylinder.surface(100, 100), xOy, y_eq_r, start_ray.trace], layout=fig_layout
)

surface_labels = ["cylinder", "xOy", "y=r", "ray"]


app = JupyterDash(__name__)
app.layout = html.Div(
    [html.H1("Geometry "),
     dcc.Graph(id="graph", figure=fig, animate=True, animation_options={"frame":{"redraw":True}}),
     html.P(id="stdout"), html.P(id="stderr", style={"color": "#f00"})]
)


@app.callback(
    Output("graph", "figure"),
    Output("stdout", "children"),
    Output("stderr", "children"),
    Input("graph", "clickData"),
    State("graph", "figure"),
)
def update_ray(click_data, figure):
    if click_data is None:
        raise PreventUpdate
    try:
        figure["layout"]["scene"]
        data = click_data["points"][0]
        point = Vector(data["x"], data["y"], data["z"])
        label = surface_labels[data["curveNumber"]]
        if label == "cylinder":
            ray = problem.get_ray(c=point)
        elif label == "xOy":
            ray = problem.get_ray(a=point)
        elif label == "y=r":
            ray = problem.get_ray(e=point)
        else:
            return no_update, str(data), "another surface"
        if ray.c.y < problem.r ** 2 / (problem.r + problem.d):
            return no_update, str(data), "point C is not seen"
        figure["data"][3] = ray.trace
        return figure, html.Div([html.P(str(data)),
                                 html.P(f"a = {ray.a}"),
                                 html.P(f"c = {ray.c}"),
                                 html.P(f"e = {ray.e}"),
                                 html.P(f"f = {ray.f}")]), no_update
    except BaseException as e:
        return no_update, str(data), html.Div([
            html.P(str(data)),
            html.Pre(
                ''.join(traceback.TracebackException.from_exception(e).format())),
            html.P('base exception')
        ])


app.run_server(mode="jupyterlab" if is_jupyterlab_running()
               else "inline", port=8052 if is_jupyterlab_running() else 8053)

In [19]:
df = problem.get_dataframe()
ax, ay, az = df['Ax'], df['Ay'], df['Az']
cx, cy, cz = df['Cx'], df['Cy'], df['Cz']
ex, ey, ez = df['Ex'], df['Ey'], df['Ez']
a2c, e2c = problem.a2c(df), problem.e2c(df)
a2cx, a2cy, a2cz = a2c['Cx'], a2c['Cy'], a2c['Cz']
e2cx, e2cy, e2cz = e2c['Cx'], e2c['Cy'], e2c['Cz']
for array in (ax, ay, az, cx, cy, cz, ex, ey, ez, a2cx, a2cy, a2cz, e2cx, e2cy, e2cz):
    array.to_numpy().shape = 100, 100

In [42]:
fig = make_subplots(
    rows=3,
    cols=1,
    specs=[
        [{"type": "scene"}],
        [{"type": "scene"}],
        [{"type": "scene"}]
    ],
    subplot_titles=["A -> Cx", "A -> Cy", "A -> Cz"],
)

fig.update_layout(height=1000)

titles = ["Cx(z)", "Cy(z)", "Cz(z)"]
for row in range(1, 4):
    fig.update_scenes(xaxis_title="Ax(x)", yaxis_title="Ay(y)", zaxis_title=titles[row - 1], row=row, col=1)

fig.add_traces(
    [
        go.Surface(x=ax.values, y=ay.values, z=cx.values, colorscale=[[0, "blue"], [1, "blue"]]),
        go.Surface(x=ax.values, y=ay.values, z=a2cx.values, colorscale=[[0, "red"], [1, "red"]]),
    ], rows=1, cols=1
)
fig.add_traces(
    [
        go.Surface(x=ax.values, y=ay.values, z=cy.values, colorscale=[[0, "blue"], [1, "blue"]]),
        go.Surface(x=ax.values, y=ay.values, z=a2cy.values, colorscale=[[0, "red"], [1, "red"]]),
    ], rows=2, cols=1
)
fig.add_traces(
    [
        go.Surface(x=ax.values, y=ay.values, z=cz.values, colorscale=[[0, "blue"], [1, "blue"]]),
        go.Surface(x=ax.values, y=ay.values, z=a2cz.values, colorscale=[[0, "red"], [1, "red"]]),
    ], rows=3, cols=1
)


app = JupyterDash(__name__)
app.layout = html.Div(
    [html.H1("Geometry "), dcc.Graph(id="graph", figure=fig), html.P(id="out")]
)

app.run_server(mode="jupyterlab" if is_jupyterlab_running() else "inline", port=8052 if is_jupyterlab_running() else 8053)

In [43]:
fig = make_subplots(
    rows=3,
    cols=1,
    specs=[
        [{"type": "scene"}],
        [{"type": "scene"}],
        [{"type": "scene"}]
    ],
    subplot_titles=["E -> Cx", "E -> Cy", "E -> Cz"],
)

fig.update_layout(height=1000)

titles = ["Cx(z)", "Cy(z)", "Cz(z)"]
for row in range(1, 4):
    fig.update_scenes(xaxis_title="Ex(x)", yaxis_title="Ez(y)", zaxis_title=titles[row - 1], row=row, col=1)

fig.add_traces(
    [
        go.Surface(x=ex.values, y=ez.values, z=cx.values, colorscale=[[0, "blue"], [1, "blue"]]),
        go.Surface(x=ex.values, y=ez.values, z=e2cx.values, colorscale=[[0, "red"], [1, "red"]]),
    ], rows=1, cols=1
)
fig.add_traces(
    [
        go.Surface(x=ex.values, y=ez.values, z=cy.values, colorscale=[[0, "blue"], [1, "blue"]]),
        go.Surface(x=ex.values, y=ez.values, z=e2cy.values, colorscale=[[0, "red"], [1, "red"]]),
    ], rows=2, cols=1
)
fig.add_traces(
    [
        go.Surface(x=ex.values, y=ez.values, z=cz.values, colorscale=[[0, "blue"], [1, "blue"]]),
        go.Surface(x=ex.values, y=ez.values, z=e2cz.values, colorscale=[[0, "red"], [1, "red"]]),
    ], rows=3, cols=1
)


app = JupyterDash(__name__)
app.layout = html.Div(
    [html.H1("Geometry "), dcc.Graph(id="graph", figure=fig), html.P(id="out")]
)

app.run_server(mode="jupyterlab" if is_jupyterlab_running() else "inline", port=8052 if is_jupyterlab_running() else 8053)

In [39]:
fig = make_subplots(
    rows=2,
    cols=1,
    specs=[
        [{"type": "scene"}],
        [{"type": "scene"}]
    ],
    subplot_titles=["C -> Ax", "C -> Ay", "C -> Az"],
)

fig.update_layout(height=1000)

titles = ["Ax(z)", "Ay(z)"]
for row in range(1, 3):
    fig.update_scenes(xaxis_title="Cx(x)", yaxis_title="Cz(y)", zaxis_title=titles[row - 1], row=row, col=1)

fig.add_traces(
    [
        go.Surface(x=cx.values, y=cz.values, z=ax.values, colorscale=[[0, "blue"], [1, "blue"]]),
    ], rows=1, cols=1
)
fig.add_traces(
    [
        go.Surface(x=cx.values, y=cz.values, z=ay.values, colorscale=[[0, "blue"], [1, "blue"]]),
    ], rows=2, cols=1
)


app = JupyterDash(__name__)
app.layout = html.Div(
    [html.H1("Geometry "), dcc.Graph(id="graph", figure=fig), html.P(id="out")]
)

app.run_server(mode="jupyterlab" if is_jupyterlab_running() else "inline", port=8052 if is_jupyterlab_running() else 8053)

# Известно A, нужно найти C
Теоретически выразить координаты $C$ через координаты $A$ не удаётся. Поэтому нужно прибегнуть к помощи машинного обучения

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

from problem import Problem, Vector, VectorRegressor

In [2]:
problem = Problem(h=2, d=1, r=1)
df = problem.get_dataframe(100, 100)

In [3]:
def get_X(v):
    x, y, z = v.x, v.y, v.z
    return (
        x * 0 + 1,
        x,
        y,
        z,
        x ** 2,
        y ** 2,
        z ** 2,
        1 / (x + 0.0001),
        1 / (y + 0.0001),
        1 / (z + 0.0001),
        x * y,
        x * z,
        y * z,
        np.arctan(y / x),
    )


f = VectorRegressor([DecisionTreeRegressor() for _ in range(3)], get_X)

In [4]:
X = df.rename(columns={'Ax': 'x', 'Ay': 'y', 'Az': 'z'})
Y = df.rename(columns={'Cx': 'x', 'Cy': 'y', 'Cz': 'z'})

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [5]:
f.fit(X_train, Y_train)

In [6]:
predicted_Y_train = f.predict(X_train)

print("Train")
for real_values, predicted_values, coord in zip(
    Y_train[['x', 'y', 'z']].values, predicted_Y_train[['x', 'y', 'z']].values, "xyz"
):
    print(f"{coord}:", MSE(real_values, predicted_values))

predicted_Y_test = f.predict(X_test)

print("Test")
for real_values, predicted_values, coord in zip(Y_test[['x', 'y', 'z']].values, predicted_Y_test[['x', 'y', 'z']].values, "xyz"):
    print(f"{coord}:", MSE(real_values, predicted_values))

Train
x: 0.0
y: 0.0
z: 4.108650548026103e-33
Test
x: 4.1246999642293303e-33
y: 4.108650548026103e-33
z: 8.706594565180416e-05


# Известно E, нужно найти C

Теоретически выразить координаты $C$ через координаты $E$ не удаётся. Поэтому нужно прибегнуть к помощи машинного обучения

In [7]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

from problem import Problem, Vector, VectorRegressor

In [8]:
problem = Problem(h=2, d=1, r=1)
df = problem.get_dataframe(100, 100)

In [9]:
X = df.rename(columns={'Ex': 'x', 'Ey': 'y', 'Ez': 'z'})
Y = df.rename(columns={'Cx': 'x', 'Cy': 'y', 'Cz': 'z'})

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [11]:
g = VectorRegressor([DecisionTreeRegressor() for _ in range(3)], get_X)
g.fit(X_train, Y_train)

In [14]:
predicted_Y_train = g.predict(X_train)

print("Train")
for real_values, predicted_values, coord in zip(
    Y_train[['x', 'y', 'z']].values, predicted_Y_train[['x', 'y', 'z']].values, "xyz"
):
    print(f"{coord}:", MSE(real_values, predicted_values))

predicted_Y_test = g.predict(X_test)

print("Test")
for real_values, predicted_values, coord in zip(Y_test[['x', 'y', 'z']].values, predicted_Y_test[['x', 'y', 'z']].values, "xyz"):
    print(f"{coord}:", MSE(real_values, predicted_values))

Train
x: 1.533553817050743e-30
y: 1.0682491424867867e-31
z: 3.3382785702712087e-31
Test
x: 2.9952062495110292e-30
y: 4.1086505480261033e-32
z: 8.706594565180416e-05
